## [ 함수 설명 ]
### 변수 설명
- 입력 변수 설명
    - data_dir : raw data(6개)가 들어있는 폴더 위치. 해당 폴더에는 raw data만 들어있어야 함.
    - save_dir : 생성된 dataframe을 저장시킬 폴더 위치.
- 출력 변수 설명 (변수로 바로 선언하는 경우를 위해)
    - df_제휴사: df3 기준으로 df1, df5, df6 merge된 dataframe
    - df_유통사: df2 기준으로 df1, df2, df5, df6 merge된 dataframe
    - LP_wo2   : LPoint without df2, df6 중 df_유통사(df2 기준)에 포함되지 않은 데이터
    - LP_wo3   : LPoint without df3, df6 중 df_제휴사(df3 기준)에 포함되지 않은 데이터

### 사용 방법
- input값인 data_dir, save_dir 지정 및 함수 변수로 입력
- csv파일로 저장이 필요하지 않은 경우, 함수 선언부 하단에 있는 csv파일 저장 코드를 지우는 것을 추천드림 (약 5~10초 소요)
- 변수로 바로 선언하지 않는다면, 함수 선언부 맨 마지막 return행 지우세요

In [1]:
import os
import pandas as pd
import numpy as np
import math
import datetime

In [2]:
# 전체 데이터 셋
def df_making(data_dir, save_dir):
    
    #휴일 구하는 함수 선언
    def get_holidays(y, m, d):
        holidays=list([[1, 1],[2, 11],[2, 12],[2, 13],[3, 1],[5, 5],[5, 19],[6, 6],[8, 15],[9, 20],[9, 21],[9, 22],[10, 3],[10, 9],[12, 25]])

        if [m, d] in holidays:  # 공휴일 날짜 안에 속하면 True
            return 1
        date = datetime.date(y, m, d)
        return int(date.isoweekday() > 5) #토, 일요일이면 True
    
    # 폴더 위치 확인
    filename = list()
    for f in os.listdir(data_dir):
        filename.append(f)

    # 파일 읽어오기
    df1 = pd.read_csv(os.path.join(data_dir, filename[0])) #고객데모정보
    df2 = pd.read_csv(os.path.join(data_dir, filename[1])) #유통사 상품 구매 내역
    df3 = pd.read_csv(os.path.join(data_dir, filename[2])) #제휴사 서비스 이용 내역
    df4 = pd.read_csv(os.path.join(data_dir, filename[3])) #유통사 상품 카테고리 마스터
    df5 = pd.read_csv(os.path.join(data_dir, filename[4])) #유통사/제휴사 점포 마스터
    df6 = pd.read_csv(os.path.join(data_dir, filename[5])) #엘페이 결제 내역

    # 컬럼명 변경
    df1.rename(columns={'cust':'고객번호', 'ma_fem_dv':'성별', 'ages':'연령대', 'zon_hlv':'거주지'}, inplace=True)
    df2.rename(columns={'cust':'고객번호', 'rct_no':'영수증', 'chnl_dv':'채널', 'cop_c':'유통사', 'pd_c':'상품코드', 'br_c':'점포코드', 'de_dt':'이용일자', 'de_hr':'이용시간', 'buy_am':'이용금액', 'buy_ct':'구매수량'}, inplace=True)
    df3.rename(columns={'cust':'고객번호', 'rct_no':'영수증', 'cop_c':'제휴사', 'br_c':'점포코드', 'de_dt':'이용일자', 'vst_dt':'방문일자', 'de_hr':'이용시간', 'buy_am':'이용금액', 'chnl_dv':'채널'}, inplace=True)
    df4.rename(columns={'pd_c':'상품코드', 'pd_nm':'소분류', 'clac_hlv_nm':'대분류', 'clac_mcls_nm':'중분류'}, inplace=True)
    df5.rename(columns={'br_c':'점포코드', 'cop_c':'제휴사', 'zon_hlv':'점포_대분류', 'zon_mcls':'점포_중분류', 'chnl_dv':'채널'}, inplace=True)
    df6.rename(columns={'cust':'고객번호', 'cop_c':'제휴사', 'rct_no':'영수증', 'de_dt':'이용일자', 'de_hr':'이용시간', 'buy_am':'이용금액', 'chnl_dv':'채널'}, inplace=True)

    ## ---------------df_제휴사 (df3 기준 merge)---------------
    print('---------------df_제휴사 (df3 기준 merge)---------------')
    # 1. df1, df3를 "고객번호" 기준으로 합
    df_제휴사 = pd.merge(df3, df1, how='inner', on='고객번호')

    if(len(df_제휴사) == len(df3)):
        print("df1과 df3이 성공적으로 merge 되었습니다.")
    else:
        print("!! df1과 df3 merge과정 확인 필요 !!")

    # 2. df_제휴사 와 df5 를 "점포코드" 기준으로 합 (이 때, df5의 제휴사는 고려하지 않았음)
    test = pd.merge(df_제휴사.drop('제휴사', axis=1), df5, how='left', on='점포코드')
    df_제휴사_test = pd.merge(df_제휴사, df5, how='left', on='점포코드')
    if(len(test) == len(df_제휴사)):
        print("df_제휴사와 df5가 성공적으로 merge 되었습니다.")
        df_제휴사 = test.copy()
        del test
    else:
        print("!! df_제휴사 + df5 merge과정 확인 필요 !!")

    # 3. df_제휴사 + LPAY 이용 여부
    # 구분코드 = 고객번호 + 이용일자 + 이용시간 + 이용금액 + 채널 + 제휴사
    df_제휴사['구분코드'] = df_제휴사['고객번호'] + df_제휴사['이용일자'].astype('str') + df_제휴사['이용시간'].astype('str') + df_제휴사['이용금액'].astype('str') + df_제휴사['채널'].astype('str') + df_제휴사['제휴사'].astype('str') 
    df6['구분코드'] = df6['고객번호'] + df6['이용일자'].astype('str') + df6['이용시간'].astype('str') + df6['이용금액'].astype('str') + df6['채널'].astype('str') + df6['제휴사'].astype('str')

    # 구분코드가 완전히 동일한 경우한 merge한다
    # merge된 경우만 확인하기 위해 inner join
    df3_6 = pd.merge(df_제휴사, df6.drop(['고객번호', '이용일자', '이용시간', '이용금액', '채널'], axis=1), how='inner', on='구분코드')
    # LPOINT 이용한 경우의 영수증 번호만 저장
    lpoint_yes = list(df3_6['영수증_x'].drop_duplicates())
    
    # 3-1. LPOINT 사용 여부 칼럼 생성
    df_제휴사['LPoint'] = [0] * len(df_제휴사)
    idx = df_제휴사.loc[df_제휴사['영수증'].isin(lpoint_yes)].index
    df_제휴사.loc[idx, 'LPoint'] = 1
    print("df_제휴사와 df6이 성공적으로 merge 되었습니다.")

    # 3-2. LPoint 사용 안한 경우 남겨두기
    lpointonly = list(set(df6['영수증']) - set(list(df3_6['영수증_y'].drop_duplicates())))
    LP_wo3 = df6.loc[df6['영수증'].isin(lpointonly)]
    print('LPoint without df3 생성 완료')

    # 4. 휴일 column 추가 ('방문일자'가 토, 일, 공휴일이면 1, 아니면 0)
    df_제휴사['휴일'] = df_제휴사['방문일자'].apply(lambda x: get_holidays(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))
    
    ## ---------------df_유통사 (df2 기준 merge)---------------
    print('---------------df_유통사 (df2 기준 merge)---------------')
    # 1. df1, df2를 "고객번호" 기준으로 merge
    df_유통사 = pd.merge(df2, df1, how='inner', on='고객번호')

    if(len(df_유통사) == len(df2)):
        print("df1과 df2이 성공적으로 merge 되었습니다.")
    else:
        print("!! df1과 df2 merge과정 확인 필요 !!")

    # 2. df_유통사, df4를 "상품코드" 기준으로 merge
    test = pd.merge(df_유통사, df4, how='inner', on='상품코드')
    if(len(test) == len(df_유통사)):
        print("df_유통사 와 df4가 성공적으로 merge 되었습니다.")
        df_유통사 = test.copy()
        del test
    else:
        print("!! df_유통사과 df4 merge과정 확인 필요 !!")

    # 3. df_유통사, df5를 "점포코드" 기준으로 merge
    test = pd.merge(df_유통사, df5, how='left', on='점포코드') #점포코드에 null값있으므로 left join
    if(len(test) == len(df_유통사)):
        print("df_유통사 와 df5가 성공적으로 merge 되었습니다.")
        df_유통사 = test.copy()
        del test
    else:
        print("!! df_유통사과 df5 merge과정 확인 필요 !!")

    # 4. df_유통사 + LPAY 이용 여부
    df_유통사['구분코드'] = df_유통사['고객번호'] + df_유통사['이용일자'].astype('str') + df_유통사['이용시간'].astype('str') + df_유통사['이용금액'].astype('int').astype('str') + df_유통사['채널'].astype('str') + df_유통사['제휴사'].astype('str')
    df6['구분코드'] = df6['고객번호'] + df6['이용일자'].astype('str') + df6['이용시간'].astype('str') + df6['이용금액'].astype('str') + df6['채널'].astype('str') + df6['제휴사'].astype('str')

    # 구분코드가 완전히 동일한 경우한 merge한다
    # merge된 경우만 확인하기 위해 inner join
    df2_6 = pd.merge(df_유통사, df6.drop(['고객번호', '이용일자', '이용시간', '이용금액', '채널'], axis=1), how='inner', on='구분코드')

    # LPOINT 이용한 경우의 영수증 번호만 저장
    lpoint_yes = list(df2_6['영수증_x'].drop_duplicates())

    # 4-1. LPOINT 사용 여부 칼럼 생성
    df_유통사['LPoint'] = [0] * len(df_유통사)
    idx = df_유통사.loc[df_유통사['영수증'].isin(lpoint_yes)].index
    df_유통사.loc[idx, 'LPoint'] = 1
    print("df_유통사와 df6이 성공적으로 merge 되었습니다.")

    # 4-2. LPoint 사용 안한 경우 남겨두기
    lpointonly = list(set(df6['영수증']) - set(list(df3_6['영수증_y'].drop_duplicates())))
    LP_wo2 = df6.loc[df6['영수증'].isin(lpointonly)]
    print('LPoint without df2 생성 완료')
    # 4. 휴일 column 추가 ('이용일자'가 토, 일, 공휴일이면 1, 아니면 0)
    df_유통사['휴일'] = df_유통사['이용일자'].apply(lambda x: get_holidays(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))
    
    #'구분코드' 열 삭제
    df_제휴사.drop('구분코드', axis=1, inplace = True)
    df_유통사.drop('구분코드', axis=1, inplace = True)
    LP_wo2.drop('구분코드', axis=1, inplace = True)
    LP_wo3.drop('구분코드', axis=1, inplace = True)
    
    ## ---------------csv 파일로 저장---------------
    df_제휴사.to_csv(os.path.join(save_dir, "df_제휴사.csv"), index=False)
    df_제휴사.to_csv(os.path.join(save_dir, "df_제휴사.csv"), index=False)
    df_유통사.to_csv(os.path.join(save_dir, "df_유통사.csv"), index=False)
    LP_wo2.to_csv(os.path.join(save_dir, "LP_wo2.csv"), index=False)
    LP_wo3.to_csv(os.path.join(save_dir, "LP_wo3.csv"), index=False)
    print('데이터 파일이 저장되었습니다.')
    
    return df_제휴사, df_유통사, LP_wo2, LP_wo3 #생성된 dataframe을 바로 이용하는 경우를 위해,

## [ 사용 예시 ]

In [4]:
import time
start = time.time()
data_dir = "../../Data/raw_data"  # 데이터 들어있는 폴더 위치
save_dir = "../../Data"           # 가공된 데이터 파일을 저장시킬 위치
df_제휴사, df_유통사, LP_wo2, LP_wo3 = df_making(data_dir, save_dir)
print('\n 소요시간 : ', round(time.time()-start), '초')

C:\Users\Seohyeon\AppData\Local\Temp/ipykernel_19172/2828033983.py:5: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  df_제휴사, df_유통사, LP_wo2, LP_wo3 = df_making(data_dir, save_dir)


---------------df_제휴사 (df3 기준 merge)---------------
df1과 df3이 성공적으로 merge 되었습니다.
df_제휴사와 df5가 성공적으로 merge 되었습니다.


KeyboardInterrupt: 

## 제휴사, 유통사를 고객별로 나누자

In [11]:
def 유통사_고객별(df):
    newdf = df[['고객번호', 'LPoint', '휴일']]
    # LPoint ratio
    유통사_LPoint = newdf[['고객번호', 'LPoint']].groupby('고객번호', as_index=False).mean()
    유통사_LPoint = 유통사_LPoint.rename(columns = {'LPoint':'lpr'})
    유통사_LPoint.sort_values(by='lpr', ascending = False)
    #유통사_LPoint.describe()
    test = pd.DataFrame()
    test = 유통사_LPoint.copy()
    print('LPoint raio 완료')
    
    # 유통사 휴일
    유통사_휴일r = newdf[['고객번호','휴일']].groupby('고객번호', as_index=False).mean()
    유통사_휴일r = 유통사_휴일r.rename(columns = {'휴일':'휴일r'})
    test = pd.merge(test, 유통사_휴일r, how='left', on='고객번호')
    print('휴일 완료')
    
    # 제휴사 비율
    test2 = df[['고객번호', '제휴사']]
    test2 = pd.get_dummies(test2, columns=['제휴사'])
    test2 = test2.groupby(['고객번호'], as_index=False).sum()
    
    if(sum(test2['size']!=(test2['제휴사_A01']+test2['제휴사_A02']+test2['제휴사_A03']+test2['제휴사_A04']+test2['제휴사_A05']))==0):
        test2['size'] = df.groupby(['고객번호'], as_index=False).size()['size'].copy()
        # 제휴사가 nan 인 경우도 있음
        test2['제휴사_A01'] = test2['제휴사_A01']/test2['size']
        test2['제휴사_A02'] = test2['제휴사_A02']/test2['size']
        test2['제휴사_A03'] = test2['제휴사_A03']/test2['size']
        test2['제휴사_A04'] = test2['제휴사_A04']/test2['size']
        test2['제휴사_A05'] = test2['제휴사_A05']/test2['size']
        test2.drop('size', axis=1, inplace=True)
        test = pd.merge(test, test2, how='left', on='고객번호')
        print('제휴사 완료')
    else:
        print('!!! 제휴사 확인 필요')
        display(test2[test2['size']!=(test2['제휴사_A01']+test2['제휴사_A02']+test2['제휴사_A03']+test2['제휴사_A04']+test2['제휴사_A05'])])
    
    # 구매 횟수
    test2['구매 횟수'] = df.groupby(['고객번호'], as_index=False).size()['size']
    df_유통사_고객별 = pd.merge(test, test2, how='left', on='고객번호')
    print('구매 횟수 완료')
    
    ##-------코드합침--------
    test = df.sort_values('고객번호')
    #test.head()

    # 채널 비율 계산
    test2 = pd.get_dummies(test[['고객번호', '채널']], columns=['채널'])
    #test2.head()

    channel = test2.groupby(['고객번호'], as_index=False).sum()
    channel['size'] = test2.groupby(['고객번호'], as_index=False).size()['size']
    
    # 계산이 제대로 되었는지 확인
    if sum(channel['size'] != (channel['채널_1'] + channel['채널_2']))==0:
        channel['채널_1'] = channel['채널_1'] / channel['size']
        channel['채널_2'] = channel['채널_2'] / channel['size']
        channel.drop(['size'], axis=1, inplace=True)
        #display(channel)
        
        df_유통사_고객별 = pd.merge(df_유통사_고객별, channel, on='고객번호')
        print('채널 완료')
    else:
        print('!! 채널 확인 필요')

    # 이용일자
    test2 = test[['고객번호', '이용일자']]
    test2['계절'] = test2['이용일자'].apply(lambda x: (int(str(x)[4:6])//4)+1)

    test2 = pd.get_dummies(test2, columns=['계절']).rename(columns={'계절_1':'봄', '계절_2':'여름', '계절_3':'가을', '계절_4':'겨울'})
    test2.drop('이용일자', axis=1, inplace=True)
    test2['봄'] = test2['봄'].astype('int')
    test2['여름'] = test2['여름'].astype('int')
    test2['가을'] = test2['가을'].astype('int')
    test2['겨울'] = test2['겨울'].astype('int')
    season = test2.groupby(['고객번호'], as_index=False).sum()
    season['size'] = test2.groupby(['고객번호'], as_index=False).size()['size']

    if sum(season['size']!=season['봄']+season['여름']+season['가을']+season['겨울'])==0:
        season['봄'] = season['봄']/season['size']
        season['여름'] = season['여름']/season['size']
        season['가을'] = season['가을']/season['size']
        season['겨울'] = season['겨울']/season['size']
        season.drop('size', axis=1, inplace=True)
        #display(season)

        df_유통사_고객별 = pd.merge(df_유통사_고객별, season, on='고객번호')
        print('이용일자 -> 계절 완료')
    else:
        print('!!! 이용일자 확인 필요')

    # 이용시간
    test2 = test[['고객번호','이용시간']]
    test2['이용시간'] = test2['이용시간'].apply(lambda x: (x//6)+1)

    test2 = pd.get_dummies(test2, columns=['이용시간']).rename(columns={'이용시간_1':'새벽', '이용시간_2':'오전', '이용시간_3':'오후', '이용시간_4':'저녁'})
    test2['새벽'] = test2['새벽'].astype('int')
    test2['오전'] = test2['오전'].astype('int')
    test2['오후'] = test2['오후'].astype('int')
    test2['저녁'] = test2['저녁'].astype('int')
    time = test2.groupby(['고객번호'], as_index=False).sum()
    time['size'] = test2.groupby(['고객번호'], as_index=False).size()['size']
    
    if sum(time['size']!=time['새벽']+time['오전']+time['오후']+time['저녁'])==0:
        time['새벽'] = time['새벽']/time['size']
        time['오전'] = time['오전']/time['size']
        time['오후'] = time['오후']/time['size']
        time['저녁'] = time['저녁']/time['size']
        time.drop('size', axis=1, inplace=True)
        #display(time)

        df_유통사_고객별 = pd.merge(df_유통사_고객별, time, on='고객번호')
        print('시간대 변경 완료')
    else:
        print('뭔가 잘못되었다 삐리삐리')

    # 이용금액
    test2 = test[['고객번호','이용금액']]
    money = test2.groupby(['고객번호'], as_index=False).sum().rename(columns={'이용금액':'이용금액_총합'})
    money['이용금액_평균'] = test2.groupby(['고객번호'], as_index=False).mean()['이용금액'].apply(lambda x: round(x, 2))
    #display(money)
    df_유통사_고객별 = pd.merge(df_유통사_고객별, money, on='고객번호')
    print('이용금액 총합/평균 계산 완료')
    
    df_유통사_고객별 = pd.merge(df, df_유통사_고객별, how='left', on='고객번호')

    return df_유통사_고객별

In [3]:
def 제휴사_고객별(df):
    import os
    import pandas as pd
    import numpy as np
    import math
    
    df = df.sort_values(by='고객번호')
    newdf = df[['고객번호', 'LPoint', '휴일']]
    # LPoint ratio
    유통사_LPoint = newdf[['고객번호', 'LPoint']].groupby('고객번호', as_index=False).mean()
    유통사_LPoint = 유통사_LPoint.rename(columns = {'LPoint':'lpr'})
    유통사_LPoint.sort_values(by='lpr', ascending = False)
    #유통사_LPoint.describe()
    test = pd.DataFrame()
    test = 유통사_LPoint.copy()
    print('LPoint raio 완료')
    
    # 휴일
    유통사_휴일r = newdf[['고객번호','휴일']].groupby('고객번호', as_index=False).mean()
    유통사_휴일r = 유통사_휴일r.rename(columns = {'휴일':'휴일r'})
    test = pd.merge(test, 유통사_휴일r, how='left', on='고객번호')
    print('휴일 완료')
    
    # 제휴사 비율
    test2 = df[['고객번호', '제휴사']]
    test2 = pd.get_dummies(test2, columns=['제휴사'])
    test2 = test2.groupby(['고객번호'], as_index=False).sum()
    test2['size'] = df.groupby(['고객번호'], as_index=False).size()['size'].copy()
    # 제휴사가 nan 인 경우도 있음
    test2['제휴사_B'] = test2['제휴사_B01']/test2['size']
    test2['제휴사_C'] = (test2['제휴사_C01']+test2['제휴사_C02'])/test2['size']
    test2['제휴사_D'] = (test2['제휴사_D01']+test2['제휴사_D02'])/test2['size']
    test2['제휴사_E'] = test2['제휴사_E01']/test2['size']
    test2.drop('size', axis=1, inplace=True)
    test2.drop(['제휴사_B01', '제휴사_C01', '제휴사_C02', '제휴사_D01', '제휴사_D02', '제휴사_E01'], axis=1, inplace=True)
    
    test = pd.merge(test, test2, how='left', on='고객번호')
    print('제휴사 완료')
    # 구매 횟수
    test2 = df.groupby(['고객번호'], as_index=False).size()
    df_유통사_고객별 = pd.merge(test, test2, how='left', on='고객번호')
    print('구매 횟수 완료')
    
    ##-------코드합침--------
    test = df.sort_values('고객번호')
    #test.head()

    # 채널 비율 계산
    test2 = pd.get_dummies(test[['고객번호', '채널']], columns=['채널'])
    #test2.head()

    channel = test2.groupby(['고객번호'], as_index=False).sum()
    channel['size'] = test2.groupby(['고객번호'], as_index=False).size()['size']
    
    # 계산이 제대로 되었는지 확인
    if sum(channel['size'] != (channel['채널_1'] + channel['채널_2']))==0:
        channel['채널_1'] = channel['채널_1'] / channel['size']
        channel['채널_2'] = channel['채널_2'] / channel['size']
        channel.drop(['size'], axis=1, inplace=True)
        #display(channel)
        
        df_유통사_고객별 = pd.merge(df_유통사_고객별, channel, on='고객번호')
        print('채널 완료')
    else:
        print('채널 잘못되었다 삐리')

    # 이용일자
    test2 = test[['고객번호', '이용일자']]
    test2['계절'] = test2['이용일자'].apply(lambda x: (int(str(x)[4:6])//4)+1)

    test2 = pd.get_dummies(test2, columns=['계절']).rename(columns={'계절_1':'봄', '계절_2':'여름', '계절_3':'가을', '계절_4':'겨울'})
    test2['봄'] = test2['봄'].astype('int')
    test2['여름'] = test2['여름'].astype('int')
    test2['가을'] = test2['가을'].astype('int')
    test2['겨울'] = test2['겨울'].astype('int')
    test2.drop('이용일자', axis=1, inplace=True)

    season = test2.groupby(['고객번호'], as_index=False).sum()
    season['size'] = test2.groupby(['고객번호'], as_index=False).size()['size']

    if sum(season['size']!=(season['봄']+season['여름']+season['가을']+season['겨울']))==0:
        season['봄'] = season['봄']/season['size']
        season['여름'] = season['여름']/season['size']
        season['가을'] = season['가을']/season['size']
        season['겨울'] = season['겨울']/season['size']
        season.drop('size', axis=1, inplace=True)
        #display(season)

        df_유통사_고객별 = pd.merge(df_유통사_고객별, season, on='고객번호')
        print('이용일자 -> 계절 완료')
    else:
        print('이용일자 잘못되었다 삐리')
        display(season[season['size']!=season['봄']+season['여름']+season['가을']+season['겨울']])

    # 이용시간
    test2 = test[['고객번호','이용시간']]
    test2['이용시간'] = test2['이용시간'].apply(lambda x: (x//6)+1)

    test2 = pd.get_dummies(test2, columns=['이용시간']).rename(columns={'이용시간_1':'새벽', '이용시간_2':'오전', '이용시간_3':'오후', '이용시간_4':'저녁'})
    test2['새벽'] = test2['새벽'].astype('int')
    test2['오전'] = test2['오전'].astype('int')
    test2['오후'] = test2['오후'].astype('int')
    test2['저녁'] = test2['저녁'].astype('int')
    time = test2.groupby(['고객번호'], as_index=False).sum()
    time['size'] = test2.groupby(['고객번호'], as_index=False).size()['size']

    if sum(time['size']!=(time['새벽']+time['오전']+time['오후']+time['저녁']))==0:
        time['새벽'] = time['새벽']/time['size']
        time['오전'] = time['오전']/time['size']
        time['오후'] = time['오후']/time['size']
        time['저녁'] = time['저녁']/time['size']
        time.drop('size', axis=1, inplace=True)
        #display(time)

        df_유통사_고객별 = pd.merge(df_유통사_고객별, time, on='고객번호')
        print('시간대 변경 완료')
    else:
        print('이용시간 뭔가 잘못되었다 삐리삐리')
        display(time[time['size']!=(time['새벽']+time['오전']+time['오후']+time['저녁'])])

    # 이용금액
    test2 = test[['고객번호','이용금액']]
    money = test2.groupby(['고객번호'], as_index=False).sum().rename(columns={'이용금액':'이용금액_총합'})
    money['이용금액_평균'] = test2.groupby(['고객번호'], as_index=False).mean()['이용금액'].apply(lambda x: round(x, 2))
    #display(money)
    df_유통사_고객별 = pd.merge(df_유통사_고객별, money, on='고객번호')
    print('이용금액 총합/평균 계산 완료')
    df_유통사_고객별 = pd.merge(df, df_유통사_고객별, how='left', on='고객번호')
    return df_유통사_고객별

In [12]:
import pandas as pd
import time
start = time.time()
df_유통사 = pd.read_csv('../../Data/df_유통사.csv')
df_유통사_고객별 = 유통사_고객별(df_유통사)
print('\n 소요시간: ', time.time()-start,'초')

LPoint raio 완료
휴일 완료
제휴사 완료
구매 횟수 완료
채널 완료


C:\Users\Seohyeon\AppData\Local\Temp/ipykernel_24564/2415198488.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test2['계절'] = test2['이용일자'].apply(lambda x: (int(str(x)[4:6])//4)+1)


이용일자 -> 계절 완료


C:\Users\Seohyeon\AppData\Local\Temp/ipykernel_24564/2415198488.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test2['이용시간'] = test2['이용시간'].apply(lambda x: (x//6)+1)


시간대 변경 완료
이용금액 총합/평균 계산 완료

 소요시간:  37.68074989318848 초


In [5]:
import pandas as pd
import time
start = time.time()
df_제휴사 = pd.read_csv('../../Data/df_제휴사.csv')
df_제휴사_고객별 = 제휴사_고객별(df_제휴사)
print('\n 소요시간: ', time.time()-start,'초')

LPoint raio 완료
휴일 완료
제휴사 완료
구매 횟수 완료
채널 완료


C:\Users\Seohyeon\AppData\Local\Temp/ipykernel_24564/1766552287.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test2['계절'] = test2['이용일자'].apply(lambda x: (int(str(x)[4:6])//4)+1)
C:\Users\Seohyeon\AppData\Local\Temp/ipykernel_24564/1766552287.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test2['이용시간'] = test2['이용시간'].apply(lambda x: (x//6)+1)


이용일자 -> 계절 완료
시간대 변경 완료
이용금액 총합/평균 계산 완료

 소요시간:  1.517399787902832 초


In [14]:
#df_제휴사_고객별.to_csv('../../Data/df_제휴사_고객별.csv', index=False)
df_유통사_고객별.to_csv('../../Data/df_유통사_고객별.csv', index=False)

In [7]:
df_유통사_고객별 = pd.read_csv('../../Data/df_유통사_고객별.csv')

In [8]:
df_유통사_고객별.columns()

,고객번호,lpr,휴일r,제휴사_A01,제휴사_A02,제휴사_A03,제휴사_A04,제휴사_A05,size,채널_1,...,봄,여름,가을,겨울,새벽,오전,오후,저녁,이용금액_총합,이용금액_평균
0,M000034966,0.000000,0.686275,0.078431,0.862745,0.000000,0.058824,0.000000,51,1.000000,...,0.294118,0.019608,0.509804,0.176471,0.000000,0.000000,0.647059,0.352941,654120.0,12825.88
1,M000136117,0.000000,0.284404,0.816514,0.064220,0.000000,0.000000,0.000000,109,0.880734,...,0.376147,0.302752,0.192661,0.128440,0.000000,0.082569,0.853211,0.064220,27373630.0,251134.22
2,M000201112,0.000000,0.571429,0.000000,0.000000,0.000000,1.000000,0.000000,21,1.000000,...,0.000000,0.428571,0.571429,0.000000,0.285714,0.000000,0.571429,0.142857,53120.0,2529.52
3,M000225114,0.000000,0.194969,0.981132,0.000000,0.018868,0.000000,0.000000,159,1.000000,...,0.157233,0.377358,0.408805,0.056604,0.000000,0.037736,0.220126,0.742138,1937700.0,12186.79
4,M000261625,0.000000,0.447761,1.000000,0.000000,0.000000,0.000000,0.000000,67,1.000000,...,0.164179,0.432836,0.373134,0.029851,0.000000,0.044776,0.686567,0.268657,6052700.0,90338.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26912,M999599111,0.000000,0.147059,0.000000,0.823529,0.000000,0.117647,0.000000,34,0.941176,...,0.058824,0.823529,0.117647,0.000000,0.000000,0.058824,0.029412,0.911765,1484022.0,43647.71
26913,M999673157,0.000000,0.375000,0.500000,0.000000,0.000000,0.500000,0.000000,24,1.000000,...,0.416667,0.125000,0.000000,0.458333,0.166667,0.041667,0.416667,0.375000,2851370.0,118807.08
26914,M999770689,0.041270,0.288889,0.069841,0.349206,0.053968,0.473016,0.000000,315,0.946032,...,0.304762,0.279365,0.377778,0.038095,0.022222,0.377778,0.282540,0.317460,1187140.0,3768.70
26915,M999849895,0.000000,0.222222,0.333333,0.597222,0.000000,0.069444,0.000000,72,1.000000,...,0.152778,0.152778,0.638889,0.055556,0.000000,0.055556,0.569444,0.375000,1103792.0,15330.44


In [9]:
df_제휴사_고객별 = pd.read_csv('../../Data/df_제휴사_고객별.csv')

In [10]:
df_제휴사_고객별

,고객번호,lpr,휴일r,제휴사_B,제휴사_C,제휴사_D,제휴사_E,size,채널_1,채널_2,봄,여름,가을,겨울,새벽,오전,오후,저녁,이용금액_총합,이용금액_평균
0,M000034966,0.000000,1.000000,0.0,0.000000,1.000000,0.0,1,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,3900,3900.00
1,M000059535,0.000000,0.000000,0.0,1.000000,0.000000,0.0,1,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,46000,46000.00
2,M000136117,0.000000,0.333333,0.0,0.000000,1.000000,0.0,15,1.000000,0.000000,0.200000,0.200000,0.466667,0.133333,0.0,0.200000,0.333333,0.466667,237400,15826.67
3,M000225114,0.000000,0.800000,0.0,0.800000,0.200000,0.0,10,1.000000,0.000000,0.000000,0.200000,0.500000,0.300000,0.0,0.200000,0.400000,0.400000,316200,31620.00
4,M000261625,0.000000,1.000000,0.0,0.000000,1.000000,0.0,1,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,23250,23250.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21764,M999708287,0.000000,0.000000,0.0,1.000000,0.000000,0.0,2,0.000000,1.000000,0.000000,0.500000,0.500000,0.000000,0.0,0.000000,0.500000,0.500000,60000,30000.00
21765,M999770689,0.066667,0.333333,0.6,0.000000,0.400000,0.0,15,1.000000,0.000000,0.200000,0.200000,0.600000,0.000000,0.0,0.200000,0.600000,0.200000,235750,15716.67
21766,M999849895,0.000000,0.000000,0.0,0.000000,1.000000,0.0,1,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,15000,15000.00
21767,M999926092,0.000000,0.142857,0.0,0.142857,0.857143,0.0,7,0.857143,0.142857,0.285714,0.285714,0.285714,0.142857,0.0,0.000000,0.571429,0.428571,129500,18500.00


In [13]:
df_제휴사_고객별[(df_제휴사_고객별['제휴사_B']+df_제휴사_고객별['제휴사_C']+df_제휴사_고객별['제휴사_D']+df_제휴사_고객별['제휴사_E'])>1]

,고객번호,lpr,휴일r,제휴사_B,제휴사_C,제휴사_D,제휴사_E,size,채널_1,채널_2,봄,여름,가을,겨울,새벽,오전,오후,저녁,이용금액_총합,이용금액_평균
10,M000621637,0.000000,0.583333,0.083333,0.000000,0.916667,0.0,12,1.000000,0.000000,0.166667,0.583333,0.166667,0.083333,0.000000,0.166667,0.750000,0.083333,552300,46025.00
11,M000705571,0.000000,0.545455,0.000000,0.545455,0.454545,0.0,11,1.000000,0.000000,0.363636,0.272727,0.272727,0.090909,0.000000,0.181818,0.818182,0.000000,198200,18018.18
13,M000871427,0.173913,0.217391,0.000000,0.434783,0.565217,0.0,23,0.782609,0.217391,0.304348,0.304348,0.391304,0.000000,0.043478,0.000000,0.434783,0.521739,203700,8856.52
19,M001248380,0.034483,0.206897,0.000000,0.586207,0.413793,0.0,29,0.413793,0.586207,0.034483,0.379310,0.448276,0.137931,0.241379,0.068966,0.310345,0.379310,181300,6251.72
37,M001932188,0.000000,0.636364,0.000000,0.454545,0.545455,0.0,11,1.000000,0.000000,0.090909,0.545455,0.181818,0.181818,0.000000,0.000000,0.727273,0.272727,124300,11300.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21716,M997283992,0.000000,0.176471,0.000000,0.764706,0.235294,0.0,17,1.000000,0.000000,0.117647,0.058824,0.823529,0.000000,0.000000,0.176471,0.588235,0.235294,279750,16455.88
21719,M997405773,0.037037,0.444444,0.000000,0.740741,0.259259,0.0,27,0.259259,0.740741,0.222222,0.481481,0.296296,0.000000,0.000000,0.185185,0.777778,0.037037,618700,22914.81
21748,M998980052,0.871795,0.461538,0.000000,0.923077,0.076923,0.0,39,0.179487,0.820513,0.025641,0.410256,0.410256,0.153846,0.000000,0.128205,0.692308,0.179487,147400,3779.49
21756,M999238704,0.000000,0.205882,0.000000,0.000000,0.970588,0.0,34,0.970588,0.029412,0.147059,0.352941,0.411765,0.088235,0.000000,0.117647,0.735294,0.147059,306800,9023.53


Index(['고객번호', '영수증', '채널', '유통사', '점포코드', '상품코드', '이용일자', '이용시간', '이용금액',
       '구매수량', '성별', '연령대', '거주지', '소분류', '대분류', '중분류', '제휴사', '점포_대분류',
       '점포_중분류', 'LPoint', '휴일', 'lpr', '휴일r', '제휴사_A01', '제휴사_A02', '제휴사_A03',
       '제휴사_A04', '제휴사_A05', 'size', '채널_1', '채널_2', '봄', '여름', '가을', '겨울',
       '새벽', '오전', '오후', '저녁', '이용금액_총합', '이용금액_평균'],
      dtype='object')